# Where are communities located that have higher vulnerability to natural disasters? (challenge 4 @ GIthub)

Who is represented in those communities? \
What is the housing make-up of those communities? \
Analysis insights and questions: What surprised you from this analysis? \
What are some limitations of the analysis? What are ways to extend the work?

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import fiona

# Table
1. [Starting](#Starting)
2. [GEOID and Shapefiles](#GEOID-and-Shapefiles)
3. [Counties Name](#Counties)
4. [Population by Race](#Population-by-Race)
5. [Population by Ages](#Population-by-Ages)
6. [Population by Gender](#Population-by-Gender)  
7. [Population by Incomes](#Population-by-Incomes) 
8. [Population without Insurance](#Population-without-Insurance) 
9. [Population with Insurance](#Population-with-Insurance)
10. [Poverty](#Poverty)
11. [Housing Cost Owner vs. Renter](#Housing-Cost-Owner-vs.-Renter)
12. [Natural Disaster: Flooding](#Natural-Disaster:-Flooding)
13. [Natural Disaster: Fire](#Natural-Disaster:-Fire)
14. [Employment](#Employment)
15. [Energy Burden](#Energy-Burden)
16. [Expected of Loss](#Expected-of-Loss)
17. [Fund](#Fund)
18. [Mortgage and Loans](#Mortgage-and-Loans)
19. [HUD (Housing Urban Development)](#HUD-(Housing-Urban-Development))
20. [Risk Management (RMP)](#Risk-Management-(RMP)) 
21. [EJ Screen](#EJ-Screen)

# Starting

In [2]:
df = pd.read_csv("data_1-FL.csv")

In [3]:
pd.set_option('display.max_columns', None) 

In [4]:
df 

,geoid,geoid_year,state,county,state_fips_code,county_fips_code,gini_index_est,gini_index_moe,emp_status_16plus_est,emp_status_16plus_moe,emp_status_tot_civ_lf_est,emp_status_tot_civ_lf_moe,num_emp_16plus_est,num_emp_16plus_moe,num_unemp_16plus_est,num_unemp_16plus_moe,num_armed_16plus_est,num_armed_16plus_moe,total_not_lf_16plus_est,total_not_lf_16plus_moe,air_toxics_cancer,air_toxics_cancer_ej,diesel_pm_ej,lead_paint_ej,ozone_ej,pm2_5_ej,superfund_prox_ej,rmp_facility_prox_ej,traffic_prox_ej,haz_waste_prox_ej,wastewater_ej,air_toxics_resp_ej,toxic_releases_ej,underground_tanks_ej,air_toxics_cancer_supp,diesel_pm_supp,lead_paint_supp,ozone_supp,pm2_5_supp,superfund_prox_supp,rmp_facility_prox_supp,traffic_prox_supp,haz_waste_prox_supp,wastewater_supp,air_toxics_resp_supp,toxic_releases_supp,underground_tanks_supp,pop_pct_2plus_races,pop_pct_white,pop_pct_black,pop_pct_american_indian,pop_pct_asian,pop_pct_native_hawaiian,pop_pct_other_race,diesel_pm,cdfi_fund_economic_distress,energy_burden,energy_burden_pctile,exp_agri_loss_rate,exp_agri_loss_rate_pctile,exp_build_loss_rate,exp_build_loss_rate_pctile,exp_pop_loss_rate,exp_pop_loss_rate_pctile,cdfi_fund_invest_areas,total_cdfi_lending,med_mortgage_loan,med_property_value,med_sba_504_loan,med_sba_7a_loan,num_mortgage_loans,num_mortgage_denials,num_mortgages_origin,num_sba_504_loans,num_sba_7a_loans,hud_opportunity_zones,ozone,air_toxics_cancer_pctile,air_toxics_cancer_ej_pctile,diesel_pm_ej_pctile,lead_paint_ej_pctile,ozone_ej_pctile,pm2_5_ej_pctile,superfund_prox_ej_pctile,rmp_facility_prox_ej_pctile,traffic_prox_ej_pctile,haz_waste_prox_ej_pctile,wastewater_ej_pctile,air_toxics_resp_ej_pctile,toxic_releases_ej_pctile,underground_tanks_ej_pctile,air_toxics_cancer_supp_pctile,diesel_pm_supp_pctile,lead_paint_supp_pctile,ozone_supp_pctile,pm2_5_supp_pctile,superfund_prox_supp_pctile,rmp_facility_supp_pctile,traffic_prox_supp_pctile,haz_waste_supp_pctile,wastewater_supp_pctile,air_toxics_resp_supp_pctile,toxic_releases_supp_pctile,underground_tanks_supp_pctile,diesel_pm_pctile,lead_paint_pctile,ozone_pctile,pm2_5_pctile,superfund_prox_pctile,rmp_facility_prox_pctile,traffic_prox_pctile,haz_waste_prox_pctile,wastewater_pctile,air_toxics_resp_pctile,toxic_releases_pctile,underground_tanks_pctile,pm2_5,superfund_prox,housing_before_1960,lead_paint,rmp_facility_prox,traffic_prox,haz_waste_prox,wastewater,lihtc_qualified_tracts,air_toxics_resp,toxic_releases_air,med_pop_age_est,med_pop_age_moe,pop_pct_5_14_est,pop_pct_5_14_moe,pop_pct_15_17_est,pop_pct_15_17_moe,pop_pct_u18_est,pop_pct_u18_moe,pop_pct_18_24_est,pop_pct_18_24_moe,pop_pct_15_44_est,pop_pct_15_44_moe,pop_pct_16plus_est,pop_pct_16plus_moe,pop_pct_18plus_est,pop_pct_18plus_moe,pop_pct_21plus_est,pop_pct_21plus_moe,pop_pct_60plus_est,pop_pct_60plus_moe,pop_pct_62plus_est,pop_pct_62plus_moe,pop_pct_65plus_est,pop_pct_65plus_moe,pop_pct_75plus_est,pop_pct_75plus_moe,pop_med_age_male_est,pop_med_age_male_moe,pop_pct_male_5_14_est,pop_pct_male_5_14_moe,pop_pct_male_15_17_est,pop_pct_male_15_17_moe,pop_pct_male_u18_est,pop_pct_male_u18_moe,pop_pct_male_18_24_est,pop_pct_male_18_24_moe,pop_pct_male_15_44_est,pop_pct_male_15_44_moe,pop_pct_male_16plus_est,pop_pct_male_16plus_moe,pop_pct_male_18plus_est,pop_pct_male_18plus_moe,pop_pct_male_21plus_est,pop_pct_male_21plus_moe,pop_pct_male_60plus_est,pop_pct_male_60plus_moe,pop_pct_male_62plus_est,pop_pct_male_62plus_moe,pop_pct_male_65plus_est,pop_pct_male_65plus_moe,pop_pct_male_75plus_est,pop_pct_male_75plus_moe,pop_med_age_fem_est,pop_med_age_fem_moe,pop_pct_fem_5_14_est,pop_pct_fem_5_14_moe,pop_pct_fem_15_17_est,pop_pct_fem_15_17_moe,pop_pct_fem_u18_est,pop_pct_fem_u18_moe,pop_pct_fem_18_24_est,pop_pct_fem_18_24_moe,pop_pct_fem_15_44_est,pop_pct_fem_15_44_moe,pop_pct_fem_16plus_est,pop_pct_fem_16plus_moe,pop_pct_fem_18plus_est,pop_pct_fem_18plus_moe,pop_pct_fem_21plus_est,pop_pct_fem_21plus_moe,pop_pct_fem_60plus_est,pop_pct_fem_60plus_moe,pop_pct_fem_62plus_est,pop_pct_fem_62pl

### Credit to Data Kind Pro Bono Data Scientist: Akshit


In [5]:
#df.columns

In [6]:
# Select all columns with integer values under 0 (i.e. -666666666, -333333333, -222222222, NaN)
# Turn all the values into 0

int_columns = df.select_dtypes(int).columns.tolist()
data_int_columns = df[int_columns]
df = df.mask(df.loc[ :,int_columns]<0 , 0)
#int_columns

In [7]:
# Double check
df

,geoid,geoid_year,state,county,state_fips_code,county_fips_code,gini_index_est,gini_index_moe,emp_status_16plus_est,emp_status_16plus_moe,emp_status_tot_civ_lf_est,emp_status_tot_civ_lf_moe,num_emp_16plus_est,num_emp_16plus_moe,num_unemp_16plus_est,num_unemp_16plus_moe,num_armed_16plus_est,num_armed_16plus_moe,total_not_lf_16plus_est,total_not_lf_16plus_moe,air_toxics_cancer,air_toxics_cancer_ej,diesel_pm_ej,lead_paint_ej,ozone_ej,pm2_5_ej,superfund_prox_ej,rmp_facility_prox_ej,traffic_prox_ej,haz_waste_prox_ej,wastewater_ej,air_toxics_resp_ej,toxic_releases_ej,underground_tanks_ej,air_toxics_cancer_supp,diesel_pm_supp,lead_paint_supp,ozone_supp,pm2_5_supp,superfund_prox_supp,rmp_facility_prox_supp,traffic_prox_supp,haz_waste_prox_supp,wastewater_supp,air_toxics_resp_supp,toxic_releases_supp,underground_tanks_supp,pop_pct_2plus_races,pop_pct_white,pop_pct_black,pop_pct_american_indian,pop_pct_asian,pop_pct_native_hawaiian,pop_pct_other_race,diesel_pm,cdfi_fund_economic_distress,energy_burden,energy_burden_pctile,exp_agri_loss_rate,exp_agri_loss_rate_pctile,exp_build_loss_rate,exp_build_loss_rate_pctile,exp_pop_loss_rate,exp_pop_loss_rate_pctile,cdfi_fund_invest_areas,total_cdfi_lending,med_mortgage_loan,med_property_value,med_sba_504_loan,med_sba_7a_loan,num_mortgage_loans,num_mortgage_denials,num_mortgages_origin,num_sba_504_loans,num_sba_7a_loans,hud_opportunity_zones,ozone,air_toxics_cancer_pctile,air_toxics_cancer_ej_pctile,diesel_pm_ej_pctile,lead_paint_ej_pctile,ozone_ej_pctile,pm2_5_ej_pctile,superfund_prox_ej_pctile,rmp_facility_prox_ej_pctile,traffic_prox_ej_pctile,haz_waste_prox_ej_pctile,wastewater_ej_pctile,air_toxics_resp_ej_pctile,toxic_releases_ej_pctile,underground_tanks_ej_pctile,air_toxics_cancer_supp_pctile,diesel_pm_supp_pctile,lead_paint_supp_pctile,ozone_supp_pctile,pm2_5_supp_pctile,superfund_prox_supp_pctile,rmp_facility_supp_pctile,traffic_prox_supp_pctile,haz_waste_supp_pctile,wastewater_supp_pctile,air_toxics_resp_supp_pctile,toxic_releases_supp_pctile,underground_tanks_supp_pctile,diesel_pm_pctile,lead_paint_pctile,ozone_pctile,pm2_5_pctile,superfund_prox_pctile,rmp_facility_prox_pctile,traffic_prox_pctile,haz_waste_prox_pctile,wastewater_pctile,air_toxics_resp_pctile,toxic_releases_pctile,underground_tanks_pctile,pm2_5,superfund_prox,housing_before_1960,lead_paint,rmp_facility_prox,traffic_prox,haz_waste_prox,wastewater,lihtc_qualified_tracts,air_toxics_resp,toxic_releases_air,med_pop_age_est,med_pop_age_moe,pop_pct_5_14_est,pop_pct_5_14_moe,pop_pct_15_17_est,pop_pct_15_17_moe,pop_pct_u18_est,pop_pct_u18_moe,pop_pct_18_24_est,pop_pct_18_24_moe,pop_pct_15_44_est,pop_pct_15_44_moe,pop_pct_16plus_est,pop_pct_16plus_moe,pop_pct_18plus_est,pop_pct_18plus_moe,pop_pct_21plus_est,pop_pct_21plus_moe,pop_pct_60plus_est,pop_pct_60plus_moe,pop_pct_62plus_est,pop_pct_62plus_moe,pop_pct_65plus_est,pop_pct_65plus_moe,pop_pct_75plus_est,pop_pct_75plus_moe,pop_med_age_male_est,pop_med_age_male_moe,pop_pct_male_5_14_est,pop_pct_male_5_14_moe,pop_pct_male_15_17_est,pop_pct_male_15_17_moe,pop_pct_male_u18_est,pop_pct_male_u18_moe,pop_pct_male_18_24_est,pop_pct_male_18_24_moe,pop_pct_male_15_44_est,pop_pct_male_15_44_moe,pop_pct_male_16plus_est,pop_pct_male_16plus_moe,pop_pct_male_18plus_est,pop_pct_male_18plus_moe,pop_pct_male_21plus_est,pop_pct_male_21plus_moe,pop_pct_male_60plus_est,pop_pct_male_60plus_moe,pop_pct_male_62plus_est,pop_pct_male_62plus_moe,pop_pct_male_65plus_est,pop_pct_male_65plus_moe,pop_pct_male_75plus_est,pop_pct_male_75plus_moe,pop_med_age_fem_est,pop_med_age_fem_moe,pop_pct_fem_5_14_est,pop_pct_fem_5_14_moe,pop_pct_fem_15_17_est,pop_pct_fem_15_17_moe,pop_pct_fem_u18_est,pop_pct_fem_u18_moe,pop_pct_fem_18_24_est,pop_pct_fem_18_24_moe,pop_pct_fem_15_44_est,pop_pct_fem_15_44_moe,pop_pct_fem_16plus_est,pop_pct_fem_16plus_moe,pop_pct_fem_18plus_est,pop_pct_fem_18plus_moe,pop_pct_fem_21plus_est,pop_pct_fem_21plus_moe,pop_pct_fem_60plus_est,pop_pct_fem_60plus_moe,pop_pct_fem_62plus_est,pop_pct_fem_62pl

# GEOID and Shapefiles

In [8]:
df_geoid = df[['geoid', 'county', 'geoid_year', 'county_fips_code', 'state_fips_code']]
df_geoid

,geoid,county,geoid_year,county_fips_code,state_fips_code
0,12001000600,0,2020,1,12
1,12001000809,0,2020,1,12
2,12001001100,0,2020,1,12
3,12001001202,0,2020,1,12
4,12001001519,0,2020,1,12
...,...,...,...,...,...
5155,12129010209,0,2020,129,12
5156,12129990000,0,2020,129,12
5157,12131950611,0,2020,131,12
5158,12133970103,0,2020,133,12


## Counties

In [9]:
counties = gpd.read_file("tl_2023_us_county//tl_2023_us_county.shp")  
counties

,STATEFP,COUNTYFP,COUNTYNS,GEOID,GEOIDFQ,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,0500000US31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477563029,10772508,+41.9158651,-096.7885168,"POLYGON ((-96.55515 41.91587, -96.55515 41.914..."
1,53,069,01513275,53069,0500000US53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680980771,61564427,+46.2946377,-123.4244583,"POLYGON ((-123.72755 46.26449, -123.72755 46.2..."
2,35,011,00933054,35011,0500000US35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016818946,29090018,+34.3592729,-104.3686961,"POLYGON ((-104.89337 34.08894, -104.89337 34.0..."
3,31,109,00835876,31109,0500000US31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169269688,22850324,+40.7835474,-096.6886584,"POLYGON ((-96.68493 40.52330, -96.69219 40.523..."
4,31,129,00835886,31129,0500000US31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645187,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27370 40.11840, -98.27374 40.122..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3230,13,123,00351260,13123,0500000US13123,Gilmer,Gilmer County,06,H1,G4020,None,None,None,A,1103893380,12276629,+34.6905232,-084.4548113,"POLYGON ((-84.30237 34.57832, -84.30329 34.577..."
3231,27,135,00659513,27135,0500000US27135,Roseau,Roseau County,06,H1,G4020,None,None,None,A,4329782924,16924046,+48.7610683,-095.8215042,"POLYGON ((-95.25857 48.88666, -95.25707 48.885..."
3232,28,089,00695768,28089,0500000US28089,Madison,Madison County,06,H1,G4020,298,27140,None,A,1849790735,72085470,+32.6343703,-090.0341603,"POLYGON ((-90.14883 32.40026, -90.14890 32.400..."
3233,48,227,01383899,48227,0500000US48227,Howard,Howard County,06,H1,G4020,None,13700,None,A,2333039148,8841781,+32.3034298,-101.4387208,"POLYGON ((-101.18138 32.21252, -101.18138 32.2..."


In [10]:
#counties.head(11)

In [11]:
counties = counties[['STATEFP', 'COUNTYFP', 'NAME', 'geometry']]
counties

,STATEFP,COUNTYFP,NAME,geometry
0,31,039,Cuming,"POLYGON ((-96.55515 41.91587, -96.55515 41.914..."
1,53,069,Wahkiakum,"POLYGON ((-123.72755 46.26449, -123.72755 46.2..."
2,35,011,De Baca,"POLYGON ((-104.89337 34.08894, -104.89337 34.0..."
3,31,109,Lancaster,"POLYGON ((-96.68493 40.52330, -96.69219 40.523..."
4,31,129,Nuckolls,"POLYGON ((-98.27370 40.11840, -98.27374 40.122..."
...,...,...,...,...
3230,13,123,Gilmer,"POLYGON ((-84.30237 34.57832, -84.30329 34.577..."
3231,27,135,Roseau,"POLYGON ((-95.25857 48.88666, -95.25707 48.885..."
3232,28,089,Madison,"POLYGON ((-90.14883 32.40026, -90.14890 32.400..."
3233,48,227,Howard,"POLYGON ((-101.18138 32.21252, -101.18138 32.2..."


In [12]:
# Only select State FP number 12; indicating Flordia
counties = counties[counties['STATEFP']== '12']
counties

,STATEFP,COUNTYFP,NAME,geometry
71,12,053,Hernando,"POLYGON ((-82.25329 28.47849, -82.25329 28.478..."
72,12,129,Wakulla,"POLYGON ((-84.16080 29.92127, -84.16104 29.915..."
73,12,131,Walton,"POLYGON ((-86.39203 30.75066, -86.39185 30.753..."
78,12,127,Volusia,"POLYGON ((-80.89930 28.79097, -80.89956 28.790..."
194,12,051,Hendry,"POLYGON ((-81.56656 26.71338, -81.56653 26.713..."
...,...,...,...,...
2985,12,039,Gadsden,"POLYGON ((-84.93252 30.60649, -84.93242 30.606..."
3107,12,045,Gulf,"POLYGON ((-85.39127 30.02816, -85.39123 30.028..."
3119,12,073,Leon,"POLYGON ((-84.38416 30.55386, -84.38408 30.553..."
3204,12,057,Hillsborough,"POLYGON ((-82.38488 27.64639, -82.38639 27.646..."


In [13]:
counties['COUNTYFP'] = counties['COUNTYFP'].astype(int)
counties['COUNTYFP']

/opt/anaconda3/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


71       53
72      129
73      131
78      127
194      51
       ... 
2985     39
3107     45
3119     73
3204     57
3208     15
Name: COUNTYFP, Length: 67, dtype: int64

In [14]:
df_geoid['county'].unique()

array([0], dtype=object)

In [15]:
df_geoid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5160 entries, 0 to 5159
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   geoid             5160 non-null   int64 
 1   county            5160 non-null   object
 2   geoid_year        5160 non-null   int64 
 3   county_fips_code  5160 non-null   int64 
 4   state_fips_code   5160 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 201.7+ KB


In [16]:
df2 = df_geoid.merge(counties, left_on='county_fips_code', right_on='COUNTYFP')
df2

,geoid,county,geoid_year,county_fips_code,state_fips_code,STATEFP,COUNTYFP,NAME,geometry
0,12001000600,0,2020,1,12,12,1,Alachua,"POLYGON ((-82.05029 29.70973, -82.05033 29.708..."
1,12001000809,0,2020,1,12,12,1,Alachua,"POLYGON ((-82.05029 29.70973, -82.05033 29.708..."
2,12001001100,0,2020,1,12,12,1,Alachua,"POLYGON ((-82.05029 29.70973, -82.05033 29.708..."
3,12001001202,0,2020,1,12,12,1,Alachua,"POLYGON ((-82.05029 29.70973, -82.05033 29.708..."
4,12001001519,0,2020,1,12,12,1,Alachua,"POLYGON ((-82.05029 29.70973, -82.05033 29.708..."
...,...,...,...,...,...,...,...,...,...
5155,12043000100,0,2020,43,12,12,43,Glades,"POLYGON ((-81.26768 27.12110, -81.25082 27.121..."
5156,12043000202,0,2020,43,12,12,43,Glades,"POLYGON ((-81.26768 27.12110, -81.25082 27.121..."
5157,12043000300,0,2020,43,12,12,43,Glades,"POLYGON ((-81.26768 27.12110, -81.25082 27.121..."
5158,12043000201,0,2020,43,12,12,43,Glades,"POLYGON ((-81.26768 27.12110, -81.25082 27.121..."


In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5160 entries, 0 to 5159
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   geoid             5160 non-null   int64   
 1   county            5160 non-null   object  
 2   geoid_year        5160 non-null   int64   
 3   county_fips_code  5160 non-null   int64   
 4   state_fips_code   5160 non-null   int64   
 5   STATEFP           5160 non-null   object  
 6   COUNTYFP          5160 non-null   int64   
 7   NAME              5160 non-null   object  
 8   geometry          5160 non-null   geometry
dtypes: geometry(1), int64(5), object(3)
memory usage: 403.1+ KB


In [18]:
# Double Checking if the merging is successful or accurate
df2['STATEFP'] = df2['STATEFP'].astype(int)

In [19]:
df3 = df2['STATEFP']

In [20]:
# If there are no differences, there is no error in merging.
df_diff = df2[df2!=df3].fillna('')
df_diff[df_diff['state_fips_code'] == None]

/var/folders/_q/dgwntvkx711d1kr0p32b0yg80000gn/T/ipykernel_76304/3696429308.py:2: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  df_diff = df2[df2!=df3].fillna('')


,geoid,county,geoid_year,county_fips_code,state_fips_code,STATEFP,COUNTYFP,NAME,geometry


In [21]:
# Completed merged shapefiles and counties. Next, let's move on to GEOID
df5 = df2[['geoid', 'county', 'geoid_year', 'COUNTYFP', 'NAME']]
df5

,geoid,county,geoid_year,COUNTYFP,NAME
0,12001000600,0,2020,1,Alachua
1,12001000809,0,2020,1,Alachua
2,12001001100,0,2020,1,Alachua
3,12001001202,0,2020,1,Alachua
4,12001001519,0,2020,1,Alachua
...,...,...,...,...,...
5155,12043000100,0,2020,43,Glades
5156,12043000202,0,2020,43,Glades
5157,12043000300,0,2020,43,Glades
5158,12043000201,0,2020,43,Glades


## GEOID - tract

In [22]:
tract = gpd.read_file("tl_2022_12_tract//tl_2022_12_tract.shp")  
tract.head(3) 

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,12,071,080203,12071080203,802.03,Census Tract 802.03,G5020,S,8741134,1791732,+26.4348430,-082.1105363,"POLYGON ((-82.14032 26.44849, -82.13991 26.449..."
1,12,071,040110,12071040110,401.10,Census Tract 401.10,G5020,S,7000293,337138,+26.6176111,-081.7900223,"POLYGON ((-81.80555 26.62380, -81.80554 26.624..."
2,12,071,040124,12071040124,401.24,Census Tract 401.24,G5020,S,6979169,32566,+26.6259190,-081.7339045,"POLYGON ((-81.75126 26.62762, -81.75095 26.627..."


In [23]:
tract = tract[['COUNTYFP', 'GEOID', 'geometry']]
tract

,COUNTYFP,GEOID,geometry
0,071,12071080203,"POLYGON ((-82.14032 26.44849, -82.13991 26.449..."
1,071,12071040110,"POLYGON ((-81.80555 26.62380, -81.80554 26.624..."
2,071,12071040124,"POLYGON ((-81.75126 26.62762, -81.75095 26.627..."
3,011,12011060303,"POLYGON ((-80.22705 26.16284, -80.22704 26.162..."
4,086,12086013600,"POLYGON ((-80.33135 25.87703, -80.33079 25.877..."
...,...,...,...
5155,011,12011070209,"POLYGON ((-80.28418 26.08834, -80.28417 26.089..."
5156,011,12011070318,"POLYGON ((-80.41705 26.10428, -80.41697 26.104..."
5157,051,12051000300,"POLYGON ((-81.56651 26.70954, -81.56628 26.709..."
5158,085,12085000603,"POLYGON ((-80.30078 27.16362, -80.30077 27.163..."


In [24]:
tract['GEOID'] = tract['GEOID'].astype(int)
tract['GEOID']

0       12071080203
1       12071040110
2       12071040124
3       12011060303
4       12086013600
           ...     
5155    12011070209
5156    12011070318
5157    12051000300
5158    12085000603
5159    12085000607
Name: GEOID, Length: 5160, dtype: int64

In [25]:
df5.head(3)

,geoid,county,geoid_year,COUNTYFP,NAME
0,12001000600,0,2020,1,Alachua
1,12001000809,0,2020,1,Alachua
2,12001001100,0,2020,1,Alachua


In [26]:
df6 = df5.merge(tract, left_on = 'geoid', right_on = 'GEOID')
df6

,geoid,county,geoid_year,COUNTYFP_x,NAME,COUNTYFP_y,GEOID,geometry
0,12001000600,0,2020,1,Alachua,001,12001000600,"POLYGON ((-82.31141 29.65210, -82.31093 29.652..."
1,12001000809,0,2020,1,Alachua,001,12001000809,"POLYGON ((-82.35586 29.63013, -82.35585 29.630..."
2,12001001100,0,2020,1,Alachua,001,12001001100,"POLYGON ((-82.38898 29.67419, -82.38897 29.674..."
3,12001001202,0,2020,1,Alachua,001,12001001202,"POLYGON ((-82.37232 29.68854, -82.37232 29.688..."
4,12001001519,0,2020,1,Alachua,001,12001001519,"POLYGON ((-82.38624 29.61730, -82.38604 29.617..."
...,...,...,...,...,...,...,...,...
5155,12043000100,0,2020,43,Glades,043,12043000100,"POLYGON ((-81.12085 26.96303, -81.12027 26.964..."
5156,12043000202,0,2020,43,Glades,043,12043000202,"POLYGON ((-81.41280 26.81245, -81.41217 26.813..."
5157,12043000300,0,2020,43,Glades,043,12043000300,"POLYGON ((-81.31509 26.93303, -81.31489 26.933..."
5158,12043000201,0,2020,43,Glades,043,12043000201,"POLYGON ((-81.56575 26.76964, -81.56574 26.772..."


# Final Transformation

In [27]:
df7 = df6.drop(columns=['COUNTYFP_x', 'COUNTYFP_y'])
df7

,geoid,county,geoid_year,NAME,GEOID,geometry
0,12001000600,0,2020,Alachua,12001000600,"POLYGON ((-82.31141 29.65210, -82.31093 29.652..."
1,12001000809,0,2020,Alachua,12001000809,"POLYGON ((-82.35586 29.63013, -82.35585 29.630..."
2,12001001100,0,2020,Alachua,12001001100,"POLYGON ((-82.38898 29.67419, -82.38897 29.674..."
3,12001001202,0,2020,Alachua,12001001202,"POLYGON ((-82.37232 29.68854, -82.37232 29.688..."
4,12001001519,0,2020,Alachua,12001001519,"POLYGON ((-82.38624 29.61730, -82.38604 29.617..."
...,...,...,...,...,...,...
5155,12043000100,0,2020,Glades,12043000100,"POLYGON ((-81.12085 26.96303, -81.12027 26.964..."
5156,12043000202,0,2020,Glades,12043000202,"POLYGON ((-81.41280 26.81245, -81.41217 26.813..."
5157,12043000300,0,2020,Glades,12043000300,"POLYGON ((-81.31509 26.93303, -81.31489 26.933..."
5158,12043000201,0,2020,Glades,12043000201,"POLYGON ((-81.56575 26.76964, -81.56574 26.772..."


In [28]:
# Creating new shapefile

#florida_geo = gpd.GeoDataFrame(df7)
#florida_geo.to_file(driver = 'ESRI Shapefile', filename= 'florida_geo.shp')

# Population by Race

In [29]:
population_race= df[['geoid','pop_pct_2plus_races','pop_pct_white', 'pop_pct_black', 'pop_pct_asian', 'pop_pct_native_hawaiian', 'pop_pct_other_race']]
population_race

,geoid,pop_pct_2plus_races,pop_pct_white,pop_pct_black,pop_pct_asian,pop_pct_native_hawaiian,pop_pct_other_race
0,12001000600,0.0,0.0,0.0,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
population_race

,geoid,pop_pct_2plus_races,pop_pct_white,pop_pct_black,pop_pct_asian,pop_pct_native_hawaiian,pop_pct_other_race
0,12001000600,0.0,0.0,0.0,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
population_race

,geoid,pop_pct_2plus_races,pop_pct_white,pop_pct_black,pop_pct_asian,pop_pct_native_hawaiian,pop_pct_other_race
0,12001000600,0.0,0.0,0.0,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
#Creating new dataset 
population_race.to_excel('population_race_florida.xlsx')

# Population by Ages

In [33]:
population_by_ages = df[['geoid','pop_pct_5_14_est',
'pop_pct_15_17_est',
'pop_pct_u18_est',
'pop_pct_18_24_est',
'pop_pct_15_44_est',
'pop_pct_16plus_est',
'pop_pct_18plus_est',
'pop_pct_21plus_est',
'pop_pct_60plus_est',
'pop_pct_62plus_est',
'pop_pct_65plus_est',
'pop_pct_75plus_est']]
population_by_ages

,geoid,pop_pct_5_14_est,pop_pct_15_17_est,pop_pct_u18_est,pop_pct_18_24_est,pop_pct_15_44_est,pop_pct_16plus_est,pop_pct_18plus_est,pop_pct_21plus_est,pop_pct_60plus_est,pop_pct_62plus_est,pop_pct_65plus_est,pop_pct_75plus_est
0,12001000600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
#Creating new dataset
population_by_ages.to_excel('population_by_ages_florida.xlsx')

# Population by Gender  

In [35]:
population_by_genders = df[['geoid','pop_med_age_male_est',
'pop_pct_male_5_14_est',
'pop_pct_male_15_17_est',
'pop_pct_male_u18_est',
'pop_pct_male_18_24_est',
'pop_pct_male_15_44_est',
'pop_pct_male_16plus_est',
'pop_pct_male_18plus_est',
'pop_pct_male_21plus_est',
'pop_pct_male_60plus_est',
'pop_pct_male_62plus_est',
'pop_pct_male_65plus_est',
'pop_pct_male_75plus_est',
                           
                           
'pop_med_age_fem_est',
'pop_pct_fem_5_14_est',
'pop_pct_fem_15_17_est',
'pop_pct_fem_u18_est',
'pop_pct_fem_18_24_est',
'pop_pct_fem_15_44_est',
'pop_pct_fem_16plus_est',
'pop_pct_fem_18plus_est',
'pop_pct_fem_21plus_est',
'pop_pct_fem_60plus_est',
'pop_pct_fem_62plus_est',
'pop_pct_fem_65plus_est',
'pop_pct_fem_75plus_est'
]]

population_by_genders

,geoid,pop_med_age_male_est,pop_pct_male_5_14_est,pop_pct_male_15_17_est,pop_pct_male_u18_est,pop_pct_male_18_24_est,pop_pct_male_15_44_est,pop_pct_male_16plus_est,pop_pct_male_18plus_est,pop_pct_male_21plus_est,pop_pct_male_60plus_est,pop_pct_male_62plus_est,pop_pct_male_65plus_est,pop_pct_male_75plus_est,pop_med_age_fem_est,pop_pct_fem_5_14_est,pop_pct_fem_15_17_est,pop_pct_fem_u18_est,pop_pct_fem_18_24_est,pop_pct_fem_15_44_est,pop_pct_fem_16plus_est,pop_pct_fem_18plus_est,pop_pct_fem_21plus_est,pop_pct_fem_60plus_est,pop_pct_fem_62plus_est,pop_pct_fem_65plus_est,pop_pct_fem_75plus_est
0,12001000600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
#Creating new dataset 
population_by_genders.to_excel('population_by_genders_florida.xlsx')

# Population by Incomes 

In [37]:
pop_income = df[['geoid','med_hh_inc_est',
'med_inc_white_est',
'med_inc_black_est',
'med_inc_ai_an_est',
'med_inc_asian_est',
'med_inc_nhopi_est',
'med_inc_other_race_est',
'med_inc_2plus_races_est',
'med_inc_hisp_est',
'med_inc_white_nh_est',
'med_inc_age_15_24_est',
'med_inc_age_25_44_est',
'med_inc_age_45_64_est',
'med_inc_age_65plus_est',
'earn_med_est',
'earn_med_male_est',
'earn_med_fem_est',
]]
pop_income

,geoid,med_hh_inc_est,med_inc_white_est,med_inc_black_est,med_inc_ai_an_est,med_inc_asian_est,med_inc_nhopi_est,med_inc_other_race_est,med_inc_2plus_races_est,med_inc_hisp_est,med_inc_white_nh_est,med_inc_age_15_24_est,med_inc_age_25_44_est,med_inc_age_45_64_est,med_inc_age_65plus_est,earn_med_est,earn_med_male_est,earn_med_fem_est
0,12001000600,26638,34643,26255,0,0,0,0,0,0,33214,0,35058,24273,26435,25949,26500,25912
1,12001000809,36052,36806,0,0,54028,0,0,0,43750,35495,35509,43411,34063,0,24808,16544,30563
2,12001001100,109459,89964,123926,0,94883,0,0,190972,155091,89500,0,115270,136250,91806,55653,61250,53635
3,12001001202,79420,85717,53053,0,0,0,0,0,0,81991,60515,79647,88828,71164,44874,46598,42287
4,12001001519,33456,41447,0,0,0,0,58333,0,32604,42500,22750,55216,80114,0,21491,27734,13355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5155,12129010209,77455,77578,77232,0,0,0,0,0,54942,78109,0,82465,78250,48523,41505,40888,45067
5156,12129990000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5157,12131950611,138583,138833,0,0,0,0,0,0,0,138833,0,0,155781,112750,77727,99519,48958
5158,12133970103,53737,54288,0,0,0,0,0,0,48125,54250,0,62454,53545,28750,34239,42500,27257


In [38]:
#pop_income.info()

In [39]:
pop_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5160 entries, 0 to 5159
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   geoid                    5160 non-null   int64
 1   med_hh_inc_est           5160 non-null   int64
 2   med_inc_white_est        5160 non-null   int64
 3   med_inc_black_est        5160 non-null   int64
 4   med_inc_ai_an_est        5160 non-null   int64
 5   med_inc_asian_est        5160 non-null   int64
 6   med_inc_nhopi_est        5160 non-null   int64
 7   med_inc_other_race_est   5160 non-null   int64
 8   med_inc_2plus_races_est  5160 non-null   int64
 9   med_inc_hisp_est         5160 non-null   int64
 10  med_inc_white_nh_est     5160 non-null   int64
 11  med_inc_age_15_24_est    5160 non-null   int64
 12  med_inc_age_25_44_est    5160 non-null   int64
 13  med_inc_age_45_64_est    5160 non-null   int64
 14  med_inc_age_65plus_est   5160 non-null   int64
 15  earn

In [40]:
#Creating New Dataset 
pop_income.to_excel('pop_income_florida.xlsx')

# Natural Disaster 

In [41]:
natural_disaster = df[['geoid','flood_risk_30yr_share', 'flood_risk_30yr_pctile', \
                      'fire_risk_30yr_share', 'fire_risk_30yr_pctile']]
natural_disaster

,geoid,flood_risk_30yr_share,flood_risk_30yr_pctile,fire_risk_30yr_share,fire_risk_30yr_pctile
0,12001000600,0.0,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0
...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0


In [42]:
#Creating new dataset 
natural_disaster.to_excel('natural_disaster_florida.xlsx')

# Population without Insurance 

In [43]:
pop_without_insurance = df[['geoid','noins_pct_est', 
'noins_u6_pct_est',
'noins_6_18_pct_est',
'noins_19_25_pct_est',
'noins_26_34_pct_est',
'noins_35_44_pct_est',
'noins_45_54_pct_est',
'noins_55_64_pct_est',
'noins_65_74_pct_est',
'noins_75plus_pct_est',
'noins_u19_pct_est',
'noins_19_64_pct_est',
'noins_65plus_pct_est',

'noins_male_pct_est',
'noins_fem_pct_est',

'noins_white_pct_est',
'noins_black_pct_est',
'noins_ai_an_pct_est',
'noins_asian_pct_est',
'noins_nhopi_pct_est',
'noins_other_race_pct_est',
'noins_2plus_races_pct_est',
'noins_hisp_pct_est'
]]

pop_without_insurance

,geoid,noins_pct_est,noins_u6_pct_est,noins_6_18_pct_est,noins_19_25_pct_est,noins_26_34_pct_est,noins_35_44_pct_est,noins_45_54_pct_est,noins_55_64_pct_est,noins_65_74_pct_est,noins_75plus_pct_est,noins_u19_pct_est,noins_19_64_pct_est,noins_65plus_pct_est,noins_male_pct_est,noins_fem_pct_est,noins_white_pct_est,noins_black_pct_est,noins_ai_an_pct_est,noins_asian_pct_est,noins_nhopi_pct_est,noins_other_race_pct_est,noins_2plus_races_pct_est,noins_hisp_pct_est
0,12001000600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
#Creating new dataset
pop_without_insurance.to_excel('pop_without_insurance_florida.xlsx')

# Population with Insurance 

In [45]:
pop_with_insurance = df[['geoid','hlthins_pop__pct_est',
'hlthins_u6_pct_est',
'hlthins_6_18_pct_est',
'hlthins_19_25_pct_est',
'hlthins_26_34_pct_est',
'hlthins_35_44_pct_est',
'hlthins_45_54_pct_est',
'hlthins_55_64_pct_est',
'hlthins_65_74_pct_est',
'hlthins_75plus_pct_est',
'hlthins_u19_pct_est',
'hlthins_19_64_pct_est',
'hlthins_65plus_pct_est',

'hlthins_male_pct_est',
'hlthins_fem_pct_est',
                        
'hlthins_white_pct_est',
'hlthins_black_pct_est',
'hlthins_ai_an_pct_est',
'hlthins_asian_pct_est',
'hlthins_nhopi_pct_est',
'hlthins_other_race_pct_est',
'hlthins_2plus_races_pct_est',
'hlthins_hisp_pct_est',
'hlthins_whitenh_pct_est']]

pop_with_insurance

,geoid,hlthins_pop__pct_est,hlthins_u6_pct_est,hlthins_6_18_pct_est,hlthins_19_25_pct_est,hlthins_26_34_pct_est,hlthins_35_44_pct_est,hlthins_45_54_pct_est,hlthins_55_64_pct_est,hlthins_65_74_pct_est,hlthins_75plus_pct_est,hlthins_u19_pct_est,hlthins_19_64_pct_est,hlthins_65plus_pct_est,hlthins_male_pct_est,hlthins_fem_pct_est,hlthins_white_pct_est,hlthins_black_pct_est,hlthins_ai_an_pct_est,hlthins_asian_pct_est,hlthins_nhopi_pct_est,hlthins_other_race_pct_est,hlthins_2plus_races_pct_est,hlthins_hisp_pct_est,hlthins_whitenh_pct_est
0,12001000600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
#Creating new dataset
pop_with_insurance.to_excel('pop_with_insurance_florida.xlsx')

# Poverty

In [47]:
poverty = df[['geoid','pov_pct_est', 
              
'pov_u18_pct_est',
'pov_u5_pct_est',
'pov_5_17_pct_est',
'pov_18_64_pct_est',
'pov_18_34_pct_est',
'pov_35_64_pct_est',
'pov_60plus_pct_est',
'pov_65plus_pct_est',
             
'pov_male_pct_est',
'pov_fem_pct_est',
             
'pov_white_pct_est',
'pov_black_pct_est',
'pov_ai_an_pct_est',
'pov_asian_pct_est',
'pov_nhopi_pct_est',
'pov_other_race_pct_est',
'pov_2plus_races_pct_est',
'pov_hisp_lat_pct_est',
'pov_white_nh_pct_est']]

poverty

,geoid,pov_pct_est,pov_u18_pct_est,pov_u5_pct_est,pov_5_17_pct_est,pov_18_64_pct_est,pov_18_34_pct_est,pov_35_64_pct_est,pov_60plus_pct_est,pov_65plus_pct_est,pov_male_pct_est,pov_fem_pct_est,pov_white_pct_est,pov_black_pct_est,pov_ai_an_pct_est,pov_asian_pct_est,pov_nhopi_pct_est,pov_other_race_pct_est,pov_2plus_races_pct_est,pov_hisp_lat_pct_est,pov_white_nh_pct_est
0,12001000600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
#Creating new dataset
poverty.to_excel('poverty_florida.xlsx')

# Housing Cost Owner vs. Renter

In [49]:
housing_cost = df[['geoid','housecost_med_est', 
   'housecost_owner_est',
   'housecost_rent_est']]

housing_cost

,geoid,housecost_med_est,housecost_owner_est,housecost_rent_est
0,12001000600,792,692,880
1,12001000809,1038,850,1070
2,12001001100,1307,1252,1398
3,12001001202,1140,1080,1224
4,12001001519,1381,0,1387
...,...,...,...,...
5155,12129010209,856,876,744
5156,12129990000,0,0,0
5157,12131950611,2750,2816,2365
5158,12133970103,827,819,903


In [50]:
#Creating new dataset
housing_cost.to_excel('housing_cost_florida.xlsx')

# Natural Disaster: Flooding 

flood_risk_30yr_share
flood_risk_30yr_pctile

In [51]:
flood = df[['geoid','flood_risk_30yr_share',
'flood_risk_30yr_pctile']]

flood

,geoid,flood_risk_30yr_share,flood_risk_30yr_pctile
0,12001000600,0.0,0.0
1,12001000809,0.0,0.0
2,12001001100,0.0,0.0
3,12001001202,0.0,0.0
4,12001001519,0.0,0.0
...,...,...,...
5155,12129010209,0.0,0.0
5156,12129990000,0.0,0.0
5157,12131950611,0.0,0.0
5158,12133970103,0.0,0.0


In [52]:
#Creating new dataset
flood.to_excel('flood_florida.xlsx')

# Natural Disaster: Fire

fire_risk_30yr_share
fire_risk_30yr_pctile

In [53]:
fire = df[['geoid','fire_risk_30yr_share',
'fire_risk_30yr_pctile',
'underground_tanks']]

fire

,geoid,fire_risk_30yr_share,fire_risk_30yr_pctile,underground_tanks
0,12001000600,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0
...,...,...,...,...
5155,12129010209,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0


In [54]:
#Creating new dataset
fire.to_excel('fire_florida.xlsx')

# Employment

In [55]:
employment = df[['geoid','emp_status_16plus_est',
'emp_status_tot_civ_lf_est']]

employment

,geoid,emp_status_16plus_est,emp_status_tot_civ_lf_est
0,12001000600,2105,2105
1,12001000809,748,748
2,12001001100,3682,3682
3,12001001202,4595,4595
4,12001001519,1362,1362
...,...,...,...
5155,12129010209,2257,2257
5156,12129990000,0,0
5157,12131950611,413,413
5158,12133970103,1582,1582


In [56]:
#Creating new dataset
employment.to_excel('employment_florida.xlsx')

# Energy Burden

In [57]:
energy_burden = df[['geoid','energy_burden',
'energy_burden_pctile']]

energy_burden

,geoid,energy_burden,energy_burden_pctile
0,12001000600,0.0,0.0
1,12001000809,0.0,0.0
2,12001001100,0.0,0.0
3,12001001202,0.0,0.0
4,12001001519,0.0,0.0
...,...,...,...
5155,12129010209,0.0,0.0
5156,12129990000,0.0,0.0
5157,12131950611,0.0,0.0
5158,12133970103,0.0,0.0


In [58]:
#Creating new dataset
energy_burden.to_excel('energy_burden_florida.xlsx')

# Expected of Loss

In [59]:
loss = df[['geoid','exp_agri_loss_rate_pctile',
'exp_build_loss_rate_pctile',
'exp_pop_loss_rate_pctile']]

loss

,geoid,exp_agri_loss_rate_pctile,exp_build_loss_rate_pctile,exp_pop_loss_rate_pctile
0,12001000600,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0
...,...,...,...,...
5155,12129010209,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0


In [60]:
loss

,geoid,exp_agri_loss_rate_pctile,exp_build_loss_rate_pctile,exp_pop_loss_rate_pctile
0,12001000600,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0
...,...,...,...,...
5155,12129010209,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0


In [61]:
#Creating new dataset
loss.to_excel('loss_florida.xlsx')

# Fund

In [62]:
fund = df[['geoid','cdfi_fund_invest_areas', 'cdfi_fund_economic_distress', 'total_cdfi_lending']]
fund

,geoid,cdfi_fund_invest_areas,cdfi_fund_economic_distress,total_cdfi_lending
0,12001000600,0,0,0.0
1,12001000809,0,0,0.0
2,12001001100,0,0,0.0
3,12001001202,0,0,0.0
4,12001001519,0,0,0.0
...,...,...,...,...
5155,12129010209,0,0,0.0
5156,12129990000,0,0,0.0
5157,12131950611,0,0,0.0
5158,12133970103,0,0,0.0


In [63]:
#Creating new dataset
fund.to_excel('fund_florida.xlsx')

# Mortgage and Loans

In [64]:
morgage_loans = df[['geoid','med_mortgage_loan',
'med_property_value',
'med_sba_504_loan',
'med_sba_7a_loan',
'num_mortgage_loans',
'num_mortgage_denials',
'num_mortgages_origin',
'num_sba_504_loans',
'num_sba_7a_loans']]

morgage_loans

,geoid,med_mortgage_loan,med_property_value,med_sba_504_loan,med_sba_7a_loan,num_mortgage_loans,num_mortgage_denials,num_mortgages_origin,num_sba_504_loans,num_sba_7a_loans
0,12001000600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
#Creating new dataset
morgage_loans.to_excel('morgage_loans_florida.xlsx')

# HUD (Housing Urban Development)

In [66]:
hud = df[['geoid','hud_opportunity_zones',
'lihtc_qualified_tracts']]
hud

,geoid,hud_opportunity_zones,lihtc_qualified_tracts
0,12001000600,1,1
1,12001000809,0,1
2,12001001100,0,0
3,12001001202,0,0
4,12001001519,0,1
...,...,...,...
5155,12129010209,0,0
5156,12129990000,0,0
5157,12131950611,0,0
5158,12133970103,0,0


In [67]:
#Creating new dataset
hud.to_excel('hud_florida.xlsx')

# Risk Management (RMP)

In [68]:
risk_management = df[['geoid','rmp_facility_supp_pctile',
'rmp_facility_prox_pctile']]

risk_management

,geoid,rmp_facility_supp_pctile,rmp_facility_prox_pctile
0,12001000600,0.0,0.0
1,12001000809,0.0,0.0
2,12001001100,0.0,0.0
3,12001001202,0.0,0.0
4,12001001519,0.0,0.0
...,...,...,...
5155,12129010209,0.0,0.0
5156,12129990000,0.0,0.0
5157,12131950611,0.0,0.0
5158,12133970103,0.0,0.0


In [69]:
#Creating new dataset
risk_management.to_excel('risk_management_florida.xlsx')

# EJ Screen

# EJ SCREEN

- ozone
- air_toxics_cancer_pctile
- air_toxics_cancer_ej_pctile
- diesel_pm_ej_pctile
- lead_paint_ej_pctile
- ozone_ej_pctile

**pm2_5_ej_pctile**
- superfund_prox_ej_pctile
- rmp_facility_prox_ej_pctile
- traffic_prox_ej_pctile
- haz_waste_prox_ej_pctile
- wastewater_ej_pctile
- air_toxics_resp_ej_pctile
- toxic_releases_ej_pctile
- underground_tanks_ej_pctile
- air_toxics_cancer_supp_pctile
- diesel_pm_supp_pctile
- lead_paint_supp_pctile
- ozone_supp_pctile

**pm2_5_supp_pctile**
- superfund_prox_supp_pctile
- rmp_facility_supp_pctile
- traffic_prox_supp_pctile
- haz_waste_supp_pctile
- wastewater_supp_pctile
- air_toxics_resp_supp_pctile
- toxic_releases_supp_pctile
- underground_tanks_supp_pctile
- diesel_pm_pctile
- lead_paint_pctile
- ozone_pctile

**pm2_5_pctile**
- superfund_prox_pctile
- rmp_facility_prox_pctile
- traffic_prox_pctile
- haz_waste_prox_pctile
- wastewater_pctile
- air_toxics_resp_pctile
- toxic_releases_pctile
- underground_tanks_pctile

**pm2_5**
- superfund_prox
- housing_before_1960
- lead_paint
- rmp_facility_prox
- traffic_prox
- haz_waste_prox
- wastewater
- lihtc_qualified_tracts
- air_toxics_resp
- toxic_releases_air

In [70]:
ejscreen =  df[[ 'geoid',
'ozone',
'air_toxics_cancer_pctile',
'air_toxics_cancer_ej_pctile',
'diesel_pm_ej_pctile',
'lead_paint_ej_pctile',
'ozone_ej_pctile',

'pm2_5_ej_pctile',
'superfund_prox_ej_pctile',
'rmp_facility_prox_ej_pctile',
'traffic_prox_ej_pctile',
'haz_waste_prox_ej_pctile',
'wastewater_ej_pctile',
'air_toxics_resp_ej_pctile',
'toxic_releases_ej_pctile',
'underground_tanks_ej_pctile',
'air_toxics_cancer_supp_pctile',
'diesel_pm_supp_pctile',
'lead_paint_supp_pctile',
'ozone_supp_pctile',

'pm2_5_supp_pctile',
'superfund_prox_supp_pctile',
'rmp_facility_supp_pctile',
'traffic_prox_supp_pctile',
'haz_waste_supp_pctile',
'wastewater_supp_pctile',
'air_toxics_resp_supp_pctile',
'toxic_releases_supp_pctile',
'underground_tanks_supp_pctile',
'diesel_pm_pctile',
'lead_paint_pctile',
'ozone_pctile',

'pm2_5_pctile',
'superfund_prox_pctile',
'rmp_facility_prox_pctile',
'traffic_prox_pctile',
'haz_waste_prox_pctile',
'wastewater_pctile',
'air_toxics_resp_pctile',
'toxic_releases_pctile',
'underground_tanks_pctile',

'pm2_5',
'superfund_prox',
'housing_before_1960',
'lead_paint',
'rmp_facility_prox',
'traffic_prox',
'haz_waste_prox',
'wastewater',
'lihtc_qualified_tracts',
'air_toxics_resp',
'toxic_releases_air']]

ejscreen

,geoid,ozone,air_toxics_cancer_pctile,air_toxics_cancer_ej_pctile,diesel_pm_ej_pctile,lead_paint_ej_pctile,ozone_ej_pctile,pm2_5_ej_pctile,superfund_prox_ej_pctile,rmp_facility_prox_ej_pctile,traffic_prox_ej_pctile,haz_waste_prox_ej_pctile,wastewater_ej_pctile,air_toxics_resp_ej_pctile,toxic_releases_ej_pctile,underground_tanks_ej_pctile,air_toxics_cancer_supp_pctile,diesel_pm_supp_pctile,lead_paint_supp_pctile,ozone_supp_pctile,pm2_5_supp_pctile,superfund_prox_supp_pctile,rmp_facility_supp_pctile,traffic_prox_supp_pctile,haz_waste_supp_pctile,wastewater_supp_pctile,air_toxics_resp_supp_pctile,toxic_releases_supp_pctile,underground_tanks_supp_pctile,diesel_pm_pctile,lead_paint_pctile,ozone_pctile,pm2_5_pctile,superfund_prox_pctile,rmp_facility_prox_pctile,traffic_prox_pctile,haz_waste_prox_pctile,wastewater_pctile,air_toxics_resp_pctile,toxic_releases_pctile,underground_tanks_pctile,pm2_5,superfund_prox,housing_before_1960,lead_paint,rmp_facility_prox,traffic_prox,haz_waste_prox,wastewater,lihtc_qualified_tracts,air_toxics_resp,toxic_releases_air
0,12001000600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
1,12001000809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
2,12001001100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
3,12001001202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
4,12001001519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5155,12129010209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
5156,12129990000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
5157,12131950611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
5158,12133970103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0


In [71]:
#Creating new dataset
ejscreen.to_excel('ejscreen_florida.xlsx')